In [ ]:
from IPython.display import display, HTML
import os
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
%run nb_core/root_dirs.ipynb
setup_syspath_disentangle(False)
%run nb_core/disentangle_imports.ipynb

In [ ]:
from disentangle.analysis.ssl_normalization_utils import get_input_normalized, get_input_unnormalized
from disentangle.data_loader.patch_index_manager import TilingMode
from disentangle.core.sampler_type import SamplerType
from disentangle.core.loss_type import LossType
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType
from disentangle.data_loader.sox2golgi_v2_rawdata_loader import Sox2GolgiV2ChannelList
from disentangle.data_loader.multifile_raw_dloader import SubDsetType
import ml_collections
from disentangle.core.tiff_reader import load_tiff
from disentangle.analysis.paper_plots import show_for_one, get_plotoutput_dir
from disentangle.analysis.ssl_normalization_utils import normalize_input
from finetunesplit.asymmetric_transforms import TransformAllChannels, VFlip, Rotate, HFlip, DeepinvTransform
import deepinv




In [ ]:
ckpt_dir = '/group/jug/ashesh/training/disentangle/2504/D21-M3-S0-L0/14'
# ckpt_dir = '/group/jug/ashesh/training/disentangle/2504/D21-M3-S0-L2/3'
# ckpt_dir = "/group/jug/ashesh/training/disentangle/2503/D17-M3-S0-L0/5"
# ckpt_dir = '/group/jug/ashesh/training/disentangle/2404/D21-M3-S0-L8/1'
assert os.path.exists(ckpt_dir)
image_size_for_grid_centers = None
mmse_count = 2
custom_image_size = None

use_selected_fpaths = None#['Test1_Slice2_a/4.nd2']
use_first_k_images = 1
tiling_mode = TilingMode.ShiftBoundary
real_input = True

save_comparative_plots =False
enable_calibration = False
batch_size = 128
num_workers = 4
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.Test 
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'


# augmentation related
aug_theta_max = 0
aug_theta_z_max = 0
aug_shift_max=0.0

# finetuningng related 
max_step_count_step1 = 20000
max_step_count_step2 =20000 
skip_pixels = 4
lr_step1 = 1e-3
lr_step2 = 1e-5
lookback = 10
k_augmentations=8
optimaization_mode = 'twostep'
k_moment_value = 3

outputdir = '/group/jug/ashesh/finetuning_results/'


In [ ]:
import numpy as np

data_t_list = None
if use_first_k_images is not None:
    data_t_list = np.arange(use_first_k_images).tolist()

assert optimaization_mode in ['onestep', 'twostep'], f"Invalid optimization mode: {optimaization_mode}"

In [ ]:
def get_dtype(ckpt_fpath):
    if os.path.isdir(ckpt_fpath):
        ckpt_fpath = ckpt_fpath[:-1] if ckpt_fpath[-1] == '/' else ckpt_fpath
    elif os.path.isfile(ckpt_fpath):
        ckpt_fpath = os.path.dirname(ckpt_fpath)
    assert ckpt_fpath[-1] != '/'
    return int(ckpt_fpath.split('/')[-2].split('-')[0][1:])

In [ ]:
dtype = get_dtype(ckpt_dir)
dtype

In [ ]:
%run nb_core/config_loader.ipynb

In [ ]:
tokens = ckpt_dir.split('/')
idx = tokens.index('disentangle')
if config.model.model_type == 25 and tokens[idx+1] == '2312':
    config.model.model_type = ModelType.LadderVAERestrictedReconstruction

In [ ]:
# from disentangle.core.lowres_merge_type import LowresMergeType


with config.unlocked():
    if 'depth3D' in config.data and config.data.depth3D > 1:
        config.data.mode_3D = True
        config.model.mode_3D = True
        
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    if dtype == DataType.HTIba1Ki67:
        config.data.subdset_type = SubDsetType.OnlyIba1P30
        config.data.empty_patch_replacement_enabled = False
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0
    if 'validtarget_random_fraction' in config.data:
        config.data.validtarget_random_fraction = None
    
    if config.data.data_type == DataType.TwoDset:
        config.model.model_type = ModelType.LadderVae
        for key in config.data.dset1:
            config.data[key] = config.data.dset1[key]
    
    if 'dump_kth_frame_prediction' in config.training:
        config.training.dump_kth_frame_prediction = None

    if 'input_is_sum' not in config.data:
        config.data.input_is_sum = False

## Replace the dataset

In [ ]:

with config.unlocked():
    if config.data.data_type in [DataType.TavernaSox2Golgi]:
        config.data.data_type = DataType.TavernaSox2GolgiV2
        config.data.subdset_type = SubDsetType.MultiChannel
        if real_input:
            print('Real Input being used')
            # all channels: ['555-647', 'GT_Cy5', 'GT_TRITC']
            config.data.channel_idx_list = [
                    Sox2GolgiV2ChannelList.GT_TRITC,Sox2GolgiV2ChannelList.GT_Cy5, Sox2GolgiV2ChannelList.GT_555_647
                ]
            config.data.input_idx = 2
        else:
            print('Real Input not being used')
            config.data.channel_idx_list = [
                    Sox2GolgiV2ChannelList.GT_TRITC,Sox2GolgiV2ChannelList.GT_Cy5
                ]
            config.data.input_idx = None

        config.data.num_channels = len(config.data.channel_idx_list)
    if use_selected_fpaths is not None:
        config.data.use_selected_fpaths = use_selected_fpaths
    config.data.target_idx_list = [0, 1]
    config.model.num_targets = len(config.data.target_idx_list)

In [ ]:
dtype = config.data.data_type

if dtype in [DataType.CustomSinosoid, DataType.CustomSinosoidThreeCurve]:
    data_dir = f'{DATA_ROOT}/sinosoid_without_test/sinosoid/'
elif dtype == DataType.OptiMEM100_014:
    data_dir = f'{DATA_ROOT}/microscopy/'
elif dtype == DataType.Prevedel_EMBL:
    data_dir = f'{DATA_ROOT}/Prevedel_EMBL/PKG_3P_dualcolor_stacks/NoAverage_NoRegistration/'
elif dtype == DataType.AllenCellMito:
    data_dir = f'{DATA_ROOT}/allencell/2017_03_08_Struct_First_Pass_Seg/AICS-11/'
elif dtype == DataType.SeparateTiffData:
    data_dir = f'{DATA_ROOT}/ventura_gigascience'
elif dtype == DataType.SemiSupBloodVesselsEMBL:
    data_dir = f'{DATA_ROOT}/EMBL_halfsupervised/Demixing_3P'
elif dtype == DataType.Pavia2VanillaSplitting:
    data_dir = f'{DATA_ROOT}/pavia2'
# elif dtype == DataType.ExpansionMicroscopyMitoTub:
    # data_dir = f'{DATA_ROOT}/expansion_microscopy_Nick/'
elif dtype == DataType.ShroffMitoEr:
    data_dir = f'{DATA_ROOT}/shrofflab/'
elif dtype == DataType.HTIba1Ki67:
    data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
elif dtype == DataType.BioSR_MRC:
    data_dir = f'{DATA_ROOT}/BioSR/'
elif dtype == DataType.ExpMicroscopyV2:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v2/datafiles/'
elif dtype == DataType.TavernaSox2Golgi:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/'
elif dtype == DataType.TavernaSox2GolgiV2:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/acquisition2/'
elif dtype == DataType.Pavia3SeqData:
    data_dir = f'{DATA_ROOT}/pavia3_sequential/'
elif dtype == DataType.NicolaData:
    data_dir = f'{DATA_ROOT}/nikola_data/20240531/'
elif dtype == DataType.Dao3ChannelWithInput:
    data_dir = f'{DATA_ROOT}/Dao4Channel/'
elif dtype == DataType.Dao3Channel:
    data_dir = f'{DATA_ROOT}/Dao3Channel/'
elif dtype == DataType.SilvioLabCSHLData:
    data_dir = f'{DATA_ROOT}/svilen_cshl2024/'
elif dtype == DataType.ExpMicroscopyV3:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v4/405_NHS_488BODIPY/'
elif dtype == DataType.Elisa3DData:
    data_dir = f'{DATA_ROOT}/Elisa3D/'

In [ ]:
# config.model.noise_model_ch1_fpath = config.model.noise_model_ch1_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')
# config.model.noise_model_ch2_fpath = config.model.noise_model_ch2_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')

In [ ]:
%run nb_core/disentangle_setup.ipynb

In [ ]:
# from disentangle.core.tiff_reader import save_tiff
# save_tiff('/group/jug/ashesh/data/sox2_one_img.tiff', val_dset.dsets[0]._data[0,...,1])

In [ ]:
if config.data.multiscale_lowres_count is not None and custom_image_size is not None:
    model.reset_for_different_output_size(custom_image_size)

In [ ]:
idx = np.random.randint(len(val_dset))
inp_tmp, tar_tmp, *_ = val_dset[idx]
ncols = len(tar_tmp)
nrows = 2
_,ax = plt.subplots(figsize=(4*ncols,4*nrows),ncols=ncols,nrows=nrows)
for i in range(min(ncols,len(inp_tmp))):
    ax[0,i].imshow(inp_tmp[i])

for channel_id in range(ncols):
    ax[1,channel_id].imshow(tar_tmp[channel_id])

In [ ]:
if data_t_list is not None:
    val_dset.reduce_data(t_list=data_t_list)

In [ ]:
# # high val dset 
# import ml_collections
# new_config = ml_collections.ConfigDict(config)
# highsnr_val_dset = None
# if 'poisson_noise_factor' in new_config.data and new_config.data.poisson_noise_factor > 0:
#     new_config.data.poisson_noise_factor = -1
#     _, highsnr_val_dset = create_dataset(new_config, data_dir, eval_datasplit_type=eval_datasplit_type,
#                                         kwargs_dict=dloader_kwargs)



In [ ]:
assert save_comparative_plots is False

## Self-supervision
Here, we now do some self-supervised finetuning. 

In [ ]:
print('Loading checkpoint from', ckpt_fpath)
checkpoint = torch.load(ckpt_fpath)
_ = model.load_state_dict(checkpoint['state_dict'], strict=False)


In [ ]:
from disentangle.analysis.stitch_prediction import stitch_predictions
from disentangle.analysis.mmse_prediction import get_dset_predictions
from deepinv.transform.projective import Homography
from finetunesplit.asymmetric_transforms import get_inverse_transforms
from disentangle.core.psnr import RangeInvariantPsnr

pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, val_dset,batch_size,
                                               num_workers=num_workers,
                                               mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                              )
tmp = np.round([x.item() for x in patch_psnr_tuple],2)
print('Patch wise PSNR, as computed during training', tmp,np.mean(tmp))

In [ ]:
mean_channels = np.mean(pred_tiled, axis=(2,3)).mean(axis=0)
std_channels = np.std(pred_tiled, axis=(2,3)).mean(axis=0)
mean_channels, std_channels

In [ ]:
from disentangle.loss.ssl_finetuning import get_stats_loss_func
stats_loss_func = get_stats_loss_func(pred_tiled, k_moment_value)
stats_loss_func(torch.Tensor(pred_tiled[:15]))
# def k_moment(data, k):
#     # data: N x C x H x W
#     dif = data - np.mean(data, axis=(2,3))[...,None, None]
#     moment = np.mean(dif**k, axis=(2,3))
#     neg_vals = moment < 0
#     moment = np.power(np.abs(moment), 1/k)
#     moment[neg_vals] = -moment[neg_vals]
#     return moment.mean(axis=0)

# k_moment(pred_tiled, 2)
# import torch
# def get_stats_loss_func():
#     mean_channels = torch.Tensor(np.mean(pred_tiled, axis=(2,3)).mean(axis=0))
#     std_channels = torch.Tensor(np.std(pred_tiled, axis=(2,3)).mean(axis=0))
#     def stats_loss_func(two_channel_prediction):
#         mean_pred = torch.mean(two_channel_prediction, dim=(2,3)).mean(dim=0)
#         std_pred = torch.std(two_channel_prediction, dim=(2,3)).mean(dim=0)
#         device = std_pred.device
#         print('est_moment', mean_pred)
#         print('actual_moment', mean_channels)
#         mean_err =  mean_channels.to(device) - mean_pred
#         mean_err = torch.clip(mean_err, min=0)
#         print('est_moment', std_pred)
#         print('actual_moment', std_channels)

#         std_err = std_channels.to(device) - std_pred
#         std_err = torch.clip(std_err, min=0)
#         mean_loss = torch.mean(mean_err)
#         std_loss = torch.mean(std_err)
#         return  mean_loss + std_loss
#     return stats_loss_func
# stats_loss_func = get_stats_loss_func()


In [ ]:
idx_list = np.where(logvar_tiled.squeeze() < -6)[0]
if len(idx_list) > 0:
    plt.imshow(val_dset[idx_list[0]][1][1])

In [ ]:
len(val_dset)

In [ ]:
print(np.quantile(rec_loss, [0,0.01,0.5, 0.9,0.99,0.999,1]).round(2))

In [ ]:
if pred_tiled.shape[-1] != val_dset.get_img_sz():
    pad = (val_dset.get_img_sz() - pred_tiled.shape[-1] )//2
    pred_tiled = np.pad(pred_tiled, ((0,0),(0,0),(pad,pad),(pad,pad)))

pred = stitch_predictions(pred_tiled,val_dset, )
if len(np.unique(logvar_tiled)) == 1:
    logvar = None
else:
    logvar = stitch_predictions(logvar_tiled,val_dset, )
pred_std = stitch_predictions(pred_std_tiled,val_dset, )

pred_original = pred.copy()

In [ ]:
pred[0].shape

In [ ]:
pred_tiled.shape

In [ ]:
is_list_prediction = isinstance(pred, list)
tar_unnorm = (val_dset._data if not is_list_prediction else [val_dset.dsets[i]._data for i in range(len(val_dset.dsets))])

In [ ]:
tar_unnorm[0].shape

In [ ]:
nTar = pred[0].shape[-1]
if "target_idx_list" in config.data and config.data.target_idx_list is not None:
    nTar =len(config.data.target_idx_list)
    # pred = pred[..., :len(config.data.target_idx_list)] if not is_list_prediction else [pred[i][..., :len(config.data.target_idx_list)] for i in range(len(pred))]
    # pred_std = pred_std[...,:len(config.data.target_idx_list)] if not is_list_prediction else [pred_std[i][..., :len(config.data.target_idx_list)] for i in range(len(pred_std))]

if 'input_idx' in config.data and config.data.input_idx is not None:
    inp_unnorm = [x[...,config.data.input_idx] for x in tar_unnorm]
    tar_unnorm = [x[...,:nTar] for x in tar_unnorm]
else:
    inp_unnorm = [x.mean(axis=-1) for x in tar_unnorm]

actual_input = normalize_input(inp_unnorm, val_dset)



### Optimal Estimation of t

In [ ]:
import numpy as np
from tqdm import tqdm
mean_psnr_arr = []
std_psnr_arr = []
t_values = np.arange(0.0,1.0, 0.05) 
for t in tqdm(t_values):
    inp_tiled = [(t *pred[i][...,0] + (1-t)*pred[i][...,1]) for i in range(len(pred))]
    psnr_values = [RangeInvariantPsnr(inp_unnorm[i]*1.0, inp_tiled[i]).item() for i in range(len(inp_unnorm))]
    mean_psnr_arr.append(np.mean(psnr_values))
    std_psnr_arr.append(np.std(psnr_values))

In [ ]:
best_idx = np.argmax(mean_psnr_arr)
best_t_estimate = t_values[best_idx]
print(f'Best t value: {best_t_estimate}')

In [ ]:
plt.plot(t_values, mean_psnr_arr)

In [ ]:
def print_ignored_pixels():
    ignored_pixels = 1
    while(pred[0,-ignored_pixels:,-ignored_pixels:,].std() ==0):
        ignored_pixels+=1
    ignored_pixels-=1
    print(f'In {pred.shape}, last {ignored_pixels} many rows and columns are all zero.')
    return ignored_pixels

In [ ]:
sep_mean, sep_std = model.data_mean["target"], model.data_std["target"]
sep_mean = sep_mean.squeeze().reshape(1, 1, 1, -1)
sep_std = sep_std.squeeze().reshape(1, 1, 1, -1)

In [ ]:
pred[0].shape, sep_mean.shape, sep_std.shape

In [ ]:
def get_unnorm_pred(pred_arr):
    if is_list_prediction:
        pred_unnorm = [pred_arr[i] * sep_std.cpu().numpy() + sep_mean.cpu().numpy() for i in range(len(pred_arr))]
    else:
        pred_unnorm = pred_arr * sep_std.cpu().numpy() + sep_mean.cpu().numpy()
    return pred_unnorm

def normalize_channels(channel_arr):
    if is_list_prediction:
        pred_unnorm = [(channel_arr[i] - sep_mean.cpu().numpy())/sep_std.cpu().numpy() for i in range(len(channel_arr))]
    else:
        pred_unnorm = (channel_arr - sep_mean.cpu().numpy())/ sep_std.cpu().numpy()
    return pred_unnorm

pred_unnorm = get_unnorm_pred(pred)

In [ ]:
from disentangle.scripts.evaluate import compute_high_snr_stats
stats_dict = compute_high_snr_stats(tar_unnorm, pred_unnorm)

In [ ]:
estimated_input = get_input_normalized(pred_original,best_t_estimate, channel_pos='last')
print('MSE-input', ' '.join([f'{np.mean((gt_tmp - pred_tmp)**2):.2f}' for gt_tmp,pred_tmp in zip(actual_input, estimated_input)]))


In [ ]:
mean_inp, std_inp = val_dset.dsets[0].get_mean_std_for_input()
mean_inp = mean_inp.reshape(-1,)[0]
std_inp = std_inp.reshape(-1,)[0]
(mean_inp, std_inp)

In [ ]:
from disentangle.analysis.ssl_plots import plot_one_sample
img_idx, hs, ws = plot_one_sample(normalize_channels(tar_unnorm), pred, inp_unnorm, config, best_t_estimate)

## SSL finetuning. 

In [ ]:
from torch.utils.data import DataLoader
from finetunesplit.loss import SSL_loss
from disentangle.loss.ssl_finetuning import finetune_two_forward_passes
from finetunesplit.asymmetric_transforms import TransformAllChannels, VFlip, Rotate, HFlip, DeepinvTransform
from deepinv.transform.projective import Homography

def reload():
    print('Loading checkpoint from', ckpt_fpath)
    checkpoint = torch.load(ckpt_fpath)
    _ = model.load_state_dict(checkpoint['state_dict'], strict=False)



In [ ]:
transform_types = {0:[VFlip(), Rotate(),HFlip()], 1:[ VFlip(), HFlip(), Rotate()]}
if aug_shift_max > 0 or aug_theta_z_max or aug_theta_max > 0:
    trans_homo = Homography(n_trans = 1, zoom_factor_min=1.0, theta_max=aug_theta_max, 
                            theta_z_max=aug_theta_z_max, 
                            skew_max=0, 
                            shift_max=aug_shift_max,
                            x_stretch_factor_min = 1,
                            y_stretch_factor_min = 1, device = model.device)
    transform_types[0].append(DeepinvTransform(trans_homo))
    transform_types[1].append(DeepinvTransform(trans_homo))


transform_all = TransformAllChannels(transform_types)

# define a learnable scalar and an offset 
factor1 = torch.nn.Parameter(torch.tensor(1.0).cuda())
offset1 = torch.nn.Parameter(torch.tensor(0.0).cuda())

factor2 = torch.nn.Parameter(torch.tensor(1.0).cuda())
offset2 = torch.nn.Parameter(torch.tensor(0.0).cuda())
mixing_ratio = torch.nn.Parameter(torch.tensor(best_t_estimate).cuda())

if optimaization_mode == 'twostep':
    optimization_params = [factor1, offset1]
    finetuning_output_dict = finetune_two_forward_passes(model, val_dset, transform_all, 
                                                        max_step_count=max_step_count_step1, 
                                                        batch_size=batch_size, 
                                                        skip_pixels=skip_pixels,
                                    scalar_params_dict={'factor1':factor1, 'offset1':offset1, 'factor2':factor2, 'offset2':offset2, 'mixing_ratio':mixing_ratio},
                                    optimization_params_dict={'lr':lr_step1, 'parameters': optimization_params},
                                    lookback=lookback,
                                    k_augmentations=k_augmentations,
                                    stats_enforcing_loss_fn=lambda x : stats_loss_func(x)
                                    )
    best_factors = finetuning_output_dict['best_factors']
    best_offsets = finetuning_output_dict['best_offsets']
elif optimaization_mode == 'onestep':
    print('Skipping the first optimization step')
    best_factors = [factor1.item(), factor2.item()]
    best_offsets = [offset1.item(), offset2.item()]

print(best_factors, best_offsets)

In [ ]:
from datetime import datetime
subdir = '_'.join(ckpt_dir.split('/')[-3:])
subsubdir = f'{optimaization_mode}_B-{batch_size}_lr-{lr_step1}-{lr_step2}_kaug-{k_augmentations}_maxstep-{max_step_count_step1}-{max_step_count_step2}_ang-{aug_theta_max}_angz-{aug_theta_z_max}_shift-{aug_shift_max}'
timesubdir = datetime.now().strftime("%Y%m%d")
direc = os.path.join(subdir,subsubdir,timesubdir)
print(direc)
if optimaization_mode == 'twostep':
    fpath = os.path.join(outputdir, direc, 'after_step1.ckpt')
    os.makedirs(os.path.dirname(fpath), exist_ok=True)
    # save the model weights.
    torch.save(model.state_dict(), fpath)
    print(f'Saved the model weights to {fpath}')
    # save the factors and offsets
    fpath = os.path.join(outputdir, direc, 'factors_offsets.txt')
    with open(fpath, 'w') as f:
        f.write(f'Factor1{best_factors[0]} Offset1{best_offsets[0]} Factor2{best_factors[1]} Offset2{best_offsets[1]}')

    

In [ ]:
from disentangle.analysis.ssl_plots import plot_finetuning_loss
if optimaization_mode == 'twostep':
    plot_finetuning_loss(finetuning_output_dict)


In [ ]:
class ScaledDataset:
    def __init__(self, dset, scalar, offset):
        self.dset = dset
        self.scalar = scalar
        self.offset = offset
    def __len__(self):
        return len(self.dset)
    def __getitem__(self, idx):
        inp, tar = self.dset[idx]
        return inp * self.scalar + self.offset, tar

In [ ]:
scaled_val_dset = ScaledDataset(val_dset, best_factors[0], best_offsets[0])

In [ ]:
pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, scaled_val_dset,batch_size,
                                               num_workers=num_workers,
                                               mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                              )
tmp = np.round([x.item() for x in patch_psnr_tuple],2)
print('Patch wise PSNR, as computed during training', tmp,np.mean(tmp))

In [ ]:
tar_unnorm[0].shape

In [ ]:
pred = stitch_predictions(pred_tiled,val_dset)
pred_unnorm = get_unnorm_pred(pred)


In [ ]:
_ = plot_one_sample(normalize_channels(tar_unnorm), pred, inp_unnorm, config, best_t_estimate, img_idx=img_idx, hs=hs, ws=ws)

In [ ]:
from disentangle.scripts.evaluate import avg_range_inv_psnr
tar0 = [x[...,0] for x in tar_unnorm]
pred0 = [x[...,0] for x in pred_unnorm]

tar1 = [x[...,1] for x in tar_unnorm]
pred1 = [x[...,1] for x in pred_unnorm]
psnr_ch0 = avg_range_inv_psnr(np.concatenate(tar0,axis=0), np.concatenate(pred0, axis=0))
psnr_ch1 = avg_range_inv_psnr(np.concatenate(tar1,axis=0), np.concatenate(pred1, axis=0))

estimated_input = get_input_normalized(pred,best_t_estimate, channel_pos='last')
mse_input_str = 'MSE-input ' +' '.join([f'{np.mean((gt_tmp - pred_tmp)**2):.2f}' for gt_tmp,pred_tmp in zip(actual_input, estimated_input)])
print(mse_input_str)
print('PSNR', psnr_ch0, psnr_ch1)

In [ ]:
avg_range_inv_psnr(np.concatenate(tar0[:1],axis=0), np.concatenate(pred0[:1], axis=0)),avg_range_inv_psnr(np.concatenate(tar1[:1],axis=0), np.concatenate(pred1[:1], axis=0))

In [ ]:
print(f'F1:{best_factors[0]:.2f} Off1:{best_offsets[0]:.2f} F2:{best_factors[1]:.2f} Off2:{best_offsets[1]:.2f}\t\t PSNR: {psnr_ch0[0]:.2f} {psnr_ch1[0]:.2f} {mse_input_str}')

In [ ]:
print(f'Target 2 Min:{tar1[0].min():.1f}, Max:{tar1[0].max():.1f}, Mean:{tar1[0].mean():.1f}, Std:{tar1[0].std():.1f}')
print(f'Pred 2 Min:{pred1[0].min():.1f}, Max:{pred1[0].max():.1f}, Mean:{pred1[0].mean():.1f}, Std:{pred1[0].std():.1f}')
print(f'Target 1 Min:{tar0[0].min():.1f}, Max:{tar0[0].max():.1f}, Mean:{tar0[0].mean():.1f}, Std:{tar0[0].std():.1f}')
print(f'Pred 1 Min:{pred0[0].min():.1f}, Max:{pred0[0].max():.1f}, Mean:{pred0[0].mean():.1f}, Std:{pred0[0].std():.1f}')

In [ ]:
# reload()
# batch_size = 128
print(max_step_count_step2)
# best_factors[0], best_offsets[0]
finetuning_output_dict = finetune_two_forward_passes(model, val_dset, transform_all, max_step_count=max_step_count_step2, 
                                                     batch_size=batch_size, skip_pixels=skip_pixels,
                                scalar_params_dict={'factor1':torch.Tensor([best_factors[0]]).cuda(), 
                                                    'offset1':torch.Tensor([best_offsets[0]]).cuda(), 
                                                    'factor2':torch.Tensor([best_factors[1]]).cuda(), 
                                                    'offset2':torch.Tensor([best_offsets[1]]).cuda(), 
                                                    'mixing_ratio':mixing_ratio},
                                optimization_params_dict={'lr':lr_step2, 
                                                        #   'parameters': model.first_bottom_up.parameters()
                                                        'parameters': model.parameters()
                                                          },
                                lookback=lookback,
                                                                k_augmentations=k_augmentations,
                                                                stats_enforcing_loss_fn=lambda x : stats_loss_func(x))

In [ ]:
from disentangle.analysis.ssl_plots import plot_finetuning_loss
plot_finetuning_loss(finetuning_output_dict, loss_rolling=10)


In [ ]:
pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, scaled_val_dset,batch_size,
                                               num_workers=num_workers,
                                               mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                              )
tmp = np.round([x.item() for x in patch_psnr_tuple],2)
print('Patch wise PSNR, as computed during training', tmp,np.mean(tmp))

In [ ]:
pred = stitch_predictions(pred_tiled,val_dset)
pred_unnorm = get_unnorm_pred(pred)

In [ ]:
_,ax = plt.subplots(figsize=(10,5),ncols=2)
ax[0].imshow(pred[0][0,...,0])
ax[1].imshow(pred[0][0,...,1])

In [ ]:
_ = plot_one_sample(normalize_channels(tar_unnorm), pred, inp_unnorm, config, best_t_estimate, img_idx=img_idx, hs=hs, ws=ws)

In [ ]:
from disentangle.scripts.evaluate import avg_range_inv_psnr
tar0 = [x[...,0] for x in tar_unnorm]
pred0 = [x[...,0] for x in pred_unnorm]

tar1 = [x[...,1] for x in tar_unnorm]
pred1 = [x[...,1] for x in pred_unnorm]
psnr_ch0 = avg_range_inv_psnr(np.concatenate(tar0,axis=0), np.concatenate(pred0, axis=0))
psnr_ch1 = avg_range_inv_psnr(np.concatenate(tar1,axis=0), np.concatenate(pred1, axis=0))

estimated_input = get_input_normalized(pred,best_t_estimate, channel_pos='last')
mse_input_str = 'MSE-input ' +' '.join([f'{np.mean((gt_tmp - pred_tmp)**2):.2f}' for gt_tmp,pred_tmp in zip(actual_input, estimated_input)])
print(mse_input_str)
print('PSNR', psnr_ch0, psnr_ch1)

In [ ]:
print(f'F1:{best_factors[0]:.2f} Off1:{best_offsets[0]:.2f} F2:{best_factors[1]:.2f} Off2:{best_offsets[1]:.2f}\t\t PSNR: {psnr_ch0[0]:.2f} {psnr_ch1[0]:.2f} {mse_input_str}')

In [ ]:
avg_range_inv_psnr(np.concatenate(tar0[:1],axis=0), np.concatenate(pred0[:1], axis=0)),avg_range_inv_psnr(np.concatenate(tar1[:1],axis=0), np.concatenate(pred1[:1], axis=0))

In [ ]:
fpath = os.path.join(outputdir, direc, 'after_step2.ckpt')
os.makedirs(os.path.dirname(fpath), exist_ok=True)
# save the model weights.
torch.save(model.state_dict(), fpath)
print(f'Saved the model weights to {fpath}')


In [ ]:
# test_img_raw = val_dset.dsets[0]._data[0,400:656,256:512,0]
# plt.imshow(test_img_raw)

In [ ]:
# from finetunesplit.aug_patch_shuffle import GridShuffle
# import torch
# import matplotlib.pyplot as plt

# trans = GridShuffle(256, 128)
# test_img = torch.Tensor(test_img_raw[None, None]*1.0)
# pred, ddict = trans(test_img)
# trans.set_params(ddict['inverse']['shuffle_order'])
# inv_pred, _ = trans(pred)
# _, ax = plt.subplots(figsize=(15,5),ncols=3)
# ax[0].imshow(test_img[0,0])
# ax[1].imshow(pred[0,0])
# ax[2].imshow(inv_pred[0,0])
# ax[0].set_title('Original')
# ax[1].set_title('Shuffled')
# ax[2].set_title('Inverse')